In [1]:
import sys

import prosperity3bt.datamodel  # noqa: F401

sys.modules["datamodel"] = sys.modules["prosperity3bt.datamodel"]

In [2]:
import os

from prosperity3bt.data import LIMITS, read_day_data
from prosperity3bt.datamodel import Order, Symbol, TradingState
from prosperity3bt.file_reader import PackageResourcesReader
from prosperity3bt.runner import TradeMatchingMode, run_backtest
from tqdm.contrib.concurrent import process_map

from prosperity3.algorithms.hybrid import DeanonymizedTradesStrategy

In [6]:
inputs = set()

file_reader = PackageResourcesReader()
for day in range(2, 5):
    data = read_day_data(file_reader, 5, day, False)
    timestamp_offset = (day - 2) * 10_000

    for _, trades_at_timestamp in sorted(data.trades.items()):
        for _, trades in trades_at_timestamp.items():
            for trade in trades:
                if trade.buyer == trade.seller or trade.price == 0:
                    continue

                inputs.add((trade.buyer, trade.symbol))

inputs = list(inputs)
len(inputs)

73

In [7]:
class Trader:
    def __init__(self, strategy: DeanonymizedTradesStrategy) -> None:
        self.strategy = strategy

    def run(self, state: TradingState) -> tuple[dict[Symbol, list[Order]], int, str]:
        orders = {}
        conversions = 0
        trader_data = ""

        strategy_orders, _ = self.strategy.run(state)
        orders[self.strategy.symbol] = strategy_orders

        return orders, conversions, trader_data


def get_profit(data: tuple[str, str]) -> tuple[float, float]:
    trader_name, symbol = data

    total_profit = 0
    max_profit = 0

    for day in range(2, 5):
        trader = Trader(DeanonymizedTradesStrategy(symbol, LIMITS[symbol], trader_name))
        result = run_backtest(trader, file_reader, 5, day, False, TradeMatchingMode.all, False, False)

        for row in reversed(result.activity_logs):
            if row.columns[2] == symbol:
                total_profit += row.columns[-1]
                break

        for row in result.activity_logs:
            if row.columns[2] == symbol:
                max_profit = max(max_profit, row.columns[-1])

    return total_profit, max_profit


profits = process_map(get_profit, inputs, max_workers=max(1, os.cpu_count() - 2))

  0%|          | 0/73 [00:00<?, ?it/s]

In [8]:
for symbol in sorted(set(symbol for _, symbol in inputs)):
    results_total = []
    results_max = []

    for (trader_name, sym), (total_profit, max_profit) in zip(inputs, profits):
        if symbol == sym:
            results_total.append((total_profit, max_profit, trader_name))
            results_max.append((max_profit, total_profit, trader_name))

    results_total = sorted(results_total, reverse=True)
    results_max = sorted(results_max, reverse=True)
    top_n = 3

    print(f"\n{symbol} top {min(top_n, len(results_total))} total profit:")
    for i, (total_profit, max_profit, trader_name) in enumerate(results_total[:top_n]):
        print(f"{i + 1}. {trader_name=} {total_profit=:,.0f} {max_profit=:,.0f}")

    print(f"\n{symbol} top {min(top_n, len(results_max))} max profit:")
    for i, (max_profit, total_profit, trader_name) in enumerate(results_max[:top_n]):
        print(f"{i + 1}. {trader_name=} {total_profit=:,.0f} {max_profit=:,.0f}")


CROISSANTS top 3 total profit:
1. trader_name='Olivia' total_profit=58,080 max_profit=25,515
2. trader_name='Camilla' total_profit=-3,808 max_profit=8,750
3. trader_name='Paris' total_profit=-319,754 max_profit=803

CROISSANTS top 3 max profit:
1. trader_name='Olivia' total_profit=58,080 max_profit=25,515
2. trader_name='Camilla' total_profit=-3,808 max_profit=8,750
3. trader_name='Paris' total_profit=-319,754 max_profit=803

DJEMBES top 3 total profit:
1. trader_name='Camilla' total_profit=751 max_profit=9,450
2. trader_name='Paris' total_profit=-92,167 max_profit=1,053
3. trader_name='Caesar' total_profit=-139,767 max_profit=723

DJEMBES top 3 max profit:
1. trader_name='Camilla' total_profit=751 max_profit=9,450
2. trader_name='Paris' total_profit=-92,167 max_profit=1,053
3. trader_name='Caesar' total_profit=-139,767 max_profit=723

JAMS top 3 total profit:
1. trader_name='Camilla' total_profit=-27,665 max_profit=20,039
2. trader_name='Paris' total_profit=-721,708 max_profit=1,697
